In [33]:
import os
import numpy as np
import matplotlib.pyplot as plt
import copy
from PIL import Image

# 1 数据集处理

## 1.1 检验元数据集正确性

In [ ]:
# 读入label中的每一条信息，并添加地址
def load_metadata(folder_list):
    tmp_lines = []
    res_lines = []
    for folder_name in folder_list:
        folder = os.path.join('data', folder_name)
        metadata_file = os.path.join(folder, 'label.txt')
        with open(metadata_file) as f:
            lines = f.readlines()
        tmp_lines.extend(list(map((folder + '\\').__add__, lines)))
    for line in tmp_lines:
        name = line.split()[0]
        if os.path.isfile(name):
            res_lines.append(line)
    return res_lines

In [ ]:
# 将一行原始信息处理成{‘图片地址’：[(边框1，关键点1),(边框2，关键点2)...]}
def load_truth(lines):
    truth = {}
    for line in lines:
        line = line.strip().split()
        name = line[0]
        if name not in truth:
            truth[name] = []
        rect = list(map(int, list(map(float, line[1:5]))))
        x = list(map(float, line[5::2]))
        y = list(map(float, line[6::2]))
        landmarks = list(zip(x, y))
        truth[name].append((rect, landmarks))
    return truth

In [ ]:
# 将显示框线的四个值转换为坐标点
def rect_trans(rect):
    r_x = np.array([rect[0],rect[0],rect[2],rect[2],rect[0]])
    r_y = np.array([rect[1],rect[3],rect[3],rect[1],rect[1]])
    return r_x,r_y

In [ ]:
# 显示指定图片,data为经过处理后的数据信息
def key_show(key,data):
    img = plt.imread(key)
    fig = plt.figure(figsize=(10,10))
    ax = fig.subplots()
    ax.axis('off')
    ax.imshow(img)
    for i in range(len(data[key])):
        rect = data[key][i][0]
        landmarks = np.array(data[key][i][1])
        r_x,r_y = rect_trans(rect)
        ax.plot(r_x,r_y,color='g',linewidth=2)
        ax.scatter(landmarks[:,0],landmarks[:,1],s=5,c='r')
    plt.show()
    
# 随机显示图片
def check_show(data):
    names = []
    for key in data:
        if key not in names:
            names.append(key)
    index = np.random.randint(0,len(names))
    name = names[index]
    key_show(name,data)

In [ ]:
folder_list = ['I', 'II']
res_lines = load_metadata(folder_list)
truth = load_truth(res_lines)

In [ ]:
check_show(truth)

## 1.2 扩大人脸框范围

In [ ]:
# 将人脸框扩大（默认0.25倍）；保证人脸框不超过图像大小
def expand_roi(rect, img_width, img_height, ratio=0.25):
    # 扩大框
    x1, y1, x2, y2 = rect[0],rect[1],rect[2],rect[3]
    width = x2 - x1 + 1
    height = y2 - y1 + 1
    padding_width = int(width * ratio)
    padding_height = int(height * ratio)
    x1 = x1 - padding_width
    y1 = y1 - padding_height
    x2 = x2 + padding_width
    y2 = y2 + padding_height
    # 保证不超过图像
    x1 = 0 if x1 < 0 else x1
    y1 = 0 if y1 < 0 else y1
    x2 = img_width - 1 if x2 >= img_width else x2
    y2 = img_height - 1 if y2 >= img_height else y2 
    rect[0],rect[1],rect[2],rect[3] = x1, y1, x2, y2

In [ ]:
def change_data_rect(data):
    for key in data:
        img = plt.imread(key)
        img_h,img_w = img.shape[:2]
        value = data[key]
        for i in range(len(value)):
            expand_roi(value[i][0],img_w,img_h)
    return data

In [ ]:
# 先深度拷贝再进行函数处理，避免原数据更改，便于调试
data_change_rect = copy.deepcopy(truth)
data_change_rect = change_data_rect(data_change_rect)

In [ ]:
check_show(data_change_rect)

In [ ]:
# 检验扩增效果
def compare_show(data1,data2):
    names = []
    for key in data1:
        if key not in names:
            names.append(key)
    index = np.random.randint(0,len(names))
    name = names[index]
    key_show(name,data1)
    key_show(name,data2)

In [ ]:
compare_show(truth,data_change_rect)

## 1.3 关键点坐标变更

In [ ]:
# 人脸关键点坐标变更 landmarks -= np.array([roi x1,roi y1])
def change_data_landmarks(data):
    delete_value1 = {}
    delete_value2 = {}
    delete_key = []
    for key in data:
        value = data[key]
        deletes1 = []
        deletes2 = []
        for i in range(len(value)):
            r = np.array([value[i][0][0],value[i][0][1]])
            w = value[i][0][2] - value[i][0][0]
            h = value[i][0][3] - value[i][0][1]
            for j in range(len(value[i][1])):
                value[i][1][j] -= r
                if value[i][1][j][0]<0 or value[i][1][j][1]<0:
                    deletes1.append(value[i])
                    break
                if value[i][1][j][0]>w or value[i][1][j][1]>h:
                    deletes2.append(value[i])
                    break
        if len(deletes1) != 0:
            delete_value1[key] = []
            for delete in deletes1:
                value.remove(delete)
                delete_value1[key].append(delete)
        if len(deletes2) != 0:
            delete_value2[key] = []
            for delete in deletes2:
                value.remove(delete)
                delete_value2[key].append(delete)
        if len(value) == 0:
            delete_key.append(key)
    for key in delete_key:
        del data[key]
            
    return data,delete_value1,delete_value2

In [ ]:
# 先深度拷贝再进行函数处理，避免原数据更改，便于调试
data_change_landmarks = copy.deepcopy(data_change_rect)
data_change_landmarks,delete_value1,delete_value2 = change_data_landmarks(data_change_landmarks)

In [ ]:
len(data_change_landmarks),len(delete_value1),len(delete_value2)

In [ ]:
# 查看删除样本情况
check_show(delete_value2)

## 1.4 生成train/test.txt

In [ ]:
# 数据类型转换
def trans_value(key,value):
    rect = ''
    for r in value[0]:
        rect += ' ' + str(r)
    landmarks = ''
    for lms in value[1]:
        landmark = ''
        for lm in lms:
            landmark += ' ' + str(lm)
        landmarks += landmark
    line = line = key+rect+landmarks
    return line

In [ ]:
# 样本个数train：test=8:2，即train有1570个，test有392个
def gen_data(data,rate=4):
    lines = []
    for key in data:
        values = data[key]
        for i in range(len(values)):
            line = trans_value(key,values[0])
            lines.append(line)
            values.remove(values[0])
    number = len(lines)
    train = lines[:int(number*(4/5))]
    test = lines[int(number*(4/5)):]
    return train,test,lines

In [ ]:
data = copy.deepcopy(data_change_landmarks)
train,test,lines = gen_data(data)
len(train),len(test),len(lines)

In [24]:
# 输出train/test.txt文件
def write_txt(data,path):
    with open(path,"w") as f:
        for d in data:
            f.write(d + '\n')
    print('File %s is ready!'%path)

In [ ]:
write_txt(train,"data\\train.txt")
write_txt(test,"data\\test.txt")

## 1.5 验证数据正确性

In [ ]:
# 读取生成的数据
def load_data(path):
    lines = []
    with open(path) as f:
        lines = f.readlines()
    data = load_truth(lines)
    return data

In [ ]:
train = load_data("data\\train.txt")
len(train)

In [ ]:
# 随机选取样本画图
def data_show(data):
    # 随机选取
    names = []
    for key in data:
        names.append(key)
    index = np.random.randint(0,len(names))
    name = names[index]
    data_key_show(name,data)
    
def data_key_show(key,data):
    # 读取原图像
    img = plt.imread(key)
    value = data[key]
    num = len(value)
    fig = plt.figure(figsize=(10,10))
    axes = fig.subplots(nrows=1,ncols=num)
    for i in range(num):
        # 画出截图头像
        crop = value[i][0]
        crop_img = img[crop[1]:crop[3],crop[0]:crop[2]]
        if num == 1:
            ax = axes
        else:
            ax = axes[i]
        ax.imshow(crop_img)
        # 画出关键点
        landmarks = np.array(value[i][1])
        ax.scatter(landmarks[:,0],landmarks[:,1],s=5,c='r')
    plt.show

In [ ]:
data_show(train)

# 2 生成stage3 非人脸样本

## 2.1 定义iou

In [2]:
def get_check_iou(rect1, rect2):
    # rect: 0-4: x1, y1, x2, y2
    left1 = rect1[0]
    top1 = rect1[1]
    right1 = rect1[2]
    bottom1 = rect1[3]
    width1 = right1 - left1 + 1
    height1 = bottom1 - top1 + 1

    left2 = rect2[0]
    top2 = rect2[1]
    right2 = rect2[2]
    bottom2 = rect2[3]
    width2 = right2 - left2 + 1
    height2 = bottom2 - top2 + 1

    w_left = max(left1, left2)
    h_left = max(top1, top2)
    w_right = min(right1, right2)
    h_right = min(bottom1, bottom2)
    inner_area = max(0, w_right - w_left + 1) * max(0, h_right - h_left + 1)

    box1_area = width1 * height1
    box2_area = width2 * height2
    iou = float(inner_area) / float(box1_area + box2_area - inner_area)
    return iou

## 2.2 生成非人脸框

### 按照正负样本1:3的比例来随机生成非人脸框。如果iou大于0.3， 则重新开始随机选取非人脸框。设定最大的随机生成次数，如果超过次数都不能生成负样本的话，输出正样本

In [3]:
import random

In [19]:
def generate_random_crops(img_w,img_h,rect,neg_gen_thre = 3,random_border =3, random_times = 10):
    neg_gen_cnt = 0
    negsample_ratio = 0.5   
    w = rect[2] - rect[0] + 1
    h = rect[3] - rect[1] + 1
    random_rect_cnt = 0
    random_rects = []
    
    while neg_gen_cnt < neg_gen_thre and random_rect_cnt < random_times :
        #print("neg_gen_cnt=",neg_gen_cnt)
        if img_h - h - random_border > 0:
            top = np.random.randint(0, img_h - h - random_border)
        else:
            top = 0
        if img_w - w - random_border > 0:
            left = np.random.randint(0, img_w - w - random_border)
        else:
            left = 0
        rect_wh = np.random.randint(min(w,h), max(w,h) + 1)
        rect_randw = np.random.randint(-3, 3)
        rect_randh = np.random.randint(-3, 3)
        right = left + rect_wh + rect_randw - 1
        bottom = top + rect_wh + rect_randh - 1

        img_rect = [0, 0, img_w - 1, img_h - 1]
        rect_img_iou = get_check_iou(rect, img_rect)
        #print("rect_img_iou=",rect_img_iou)
        if rect_img_iou > negsample_ratio:
            random_rect_cnt += 1
            break
        random_rect = [left, top, right, bottom]
        #print("random_rect=",random_rect)
        iou = get_check_iou(random_rect, rect)
        #print("iou=",iou)

        if iou < 0.3:
            #print("iou=",iou)
            random_rects.append(random_rect)
            neg_gen_cnt += 1
        else:
            random_rect_cnt += 1
            #print("random_rect_cnt=",random_rect_cnt)
    return random_rects

## 2.3 读取正样本，生成3份负样本

In [28]:
def write_error(line):
    error_list = []
    line = line.strip().split()
    img_w, img_h = (Image.open(line[0])).size
    rect = list(map(int, line[1:5]))
    error = generate_random_crops(img_w, img_h,rect, neg_gen_thre = 3,random_border = 3,random_times = 10)
    if error != []:
        for i in range (len(error)):
            s = line[0] +" "+ str(error[i][0])+" "+str(error[i][1])+" "+str(error[i][2])+" "+str(error[i][3]) + " 0 "
            error_list.append(s)
    else:
        error_list = []
    return error_list

## 2.4 读取stage1 的txt文件，生成全部正负样本,写入新的txt中

In [29]:
def gen_stage3_data(path, data):
    data_file = path + "\\" + data

    with open(data_file) as f:
        lines = f.readlines()
        
    new_lines = []
    for line in lines:
        new_lines.append(line[:-1] + " 1 ")
        error_list = write_error(line)
        if error_list != []:
            for i in range (len(error_list)):
                new_lines.append(error_list[i])
            
    print(len(new_lines))

    
    return new_lines

In [30]:
new_train = gen_stage3_data("data", "train.txt")

7250


In [31]:
new_test = gen_stage3_data("data", "test.txt")

1895


In [32]:
write_txt(new_train,"data//stage_3_train.txt")
write_txt(new_test,"data//stage_3_test.txt")

File data//stage_3_train.txt is ready!
File data//stage_3_test.txt is ready!
